In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: maiquanh. Use `wandb login --relogin` to force relogin


True

In [2]:
import numpy as np
import pandas as pd
import torch.nn as nn

import pickle
import torch
import time
import copy
import sys
import time
import os
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [3]:
import yaml
config_wbd = {}
with open('sweep.yml', 'r') as f:
    config_wdb = yaml.safe_load(f)

In [4]:
config_wdb

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'val_acc'},
 'parameters': {'batch_size': {'value': 128},
  'd_hidden': {'value': 3},
  'kernel_size_1': {'distribution': 'int_uniform', 'min': 50, 'max': 100},
  'kernel_size_2': {'distribution': 'int_uniform', 'min': 3, 'max': 11},
  'maxpool_1': {'distribution': 'int_uniform', 'min': 3, 'max': 8},
  'maxpool_2': {'distribution': 'int_uniform', 'min': 3, 'max': 8},
  'stride_1': {'distribution': 'int_uniform', 'min': 3, 'max': 9},
  'stride_2': {'distribution': 'int_uniform', 'min': 3, 'max': 9},
  'num_epoch': {'distribution': 'int_uniform', 'min': 80, 'max': 100},
  'num_filters': {'distribution': 'int_uniform', 'min': 16, 'max': 64},
  'num_layers': {'distribution': 'int_uniform', 'min': 2, 'max': 4},
  'lr': {'distribution': 'log_uniform_values', 'min': 0.0001, 'max': 0.005},
  'weight_decay': {'distribution': 'log_uniform_values',
   'min': 0.0001,
   'max': 0.0003},
  'dropout': {'distribution': 'uniform', 'min': 0.1, 

In [5]:
#dataloader
# import numpy as np
# import pickle
# from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, config, path, mode='train'):
        super().__init__()
        self.config = config
        self.mode = mode
        self.path = path
        self.load_dataset()
    
    def __len__(self):
        return self.data_.shape[0]
    
    def __getitem__(self, index):
        if self.mode == 'test':
            return self.data_[index]
        else:
            return {'input':self.data_[index], 
                    'labels': self.labels[index]}
    
    def load_dataset(self):
        data = {}
        path_ = self.config['data_dir'] + self.path
        if self.mode != 'test':
            with np.load(path_, 'rb') as f:
                data['train'] = f['data']
                data['labels'] = f['labels']
            
            self.data_ = data['train'].transpose(0,2,1)
            self.labels = data['labels']
        else:
            
            with np.load(path_, 'rb') as f:
                data = f['data']
            
            self.data_ = data.tranpose(0,2,1)
    def get_lenght(self):
        return self.lenght
            

# Model

In [6]:
#model
# import torch.nn as nn
class Conv_lstm(nn.Module):
    def __init__(self, l_data, num_filters, kernel_size, strides, maxpooling, d_hidden, num_layers, dropout=0.1):
        super().__init__()
        self.l_data = l_data
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.d_hidden = d_hidden
        self.num_layers = num_layers
        self.dropout = dropout

        self.conv1 = nn.Conv1d(1, self.num_filters, self.kernel_size[0],self.strides[0])
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.zeros_(self.conv1.bias)
        
        d_model1 = int((self.l_data - self.kernel_size[0])/self.strides[0] + 1)
        d_model2 = int((d_model1 - maxpooling[0])/maxpooling[0] + 1)
        
        self.conv2 = nn.Conv1d(self.num_filters, self.num_filters, self.kernel_size[1], self.strides[1])
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.zeros_(self.conv2.bias)
        
        d_model3 = int((d_model2 - self.kernel_size[1])/self.strides[1] + 1)
        
        self.f_activation1 = nn.ReLU()
        self.f_activation2 = nn.ReLU()
        
        self.batchNorm1 = nn.BatchNorm1d(num_features=self.num_filters) # momentum=0.99
        self.batchNorm2 = nn.BatchNorm1d(num_features=self.num_filters)
        
        self.maxpooling1 = nn.MaxPool1d(maxpooling[0])
        self.maxpooling2 = nn.MaxPool1d(maxpooling[1])
        
        d_model4 = int((d_model3 - maxpooling[1])/maxpooling[1] + 1)
        
        self.lstm = nn.LSTM(d_model4, self.d_hidden, self.num_layers, batch_first=True, dropout=self.dropout)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        
        #layer1
        self.conv1_out = self.conv1(x)
        self.conv1relu_out = self.f_activation1(self.conv1_out)
        
        #layer2
        self.maxpooling1_out = self.maxpooling1(self.conv1relu_out)
        self.batchNorm1_out = self.batchNorm1(self.maxpooling1_out )
        
        self.conv2_out = self.conv2(self.maxpooling1_out)
        self.conv2relu_out = self.f_activation2(self.conv2_out)
        
        #layer3
        self.maxpooling2_out = self.maxpooling2(self.conv2relu_out)
        self.batchNorm2_out = self.batchNorm2(self.maxpooling2_out)
        
        #layer4
        h_c = self.init_hidden(len(self.batchNorm2_out))
        self.out, (_, _) = self.lstm(self.batchNorm2_out, h_c)
        return self.softmax(self.out[:, -1, :])
            
    def init_hidden(self, batch_size):
        weight = next(self.lstm.parameters()).data
        
        hidden = weight.new(self.num_layers, batch_size, self.d_hidden).zero_()        
        cell = weight.new(self.num_layers, batch_size, self.d_hidden).zero_()
        
        return (hidden, cell)
    def get_output_per_layer(self):
        return {
                'conv1_relu': self.conv1relu_out,
                'batchNorm1': self.batchNorm1_out,
                'maxpooling1': self.maxpooling1_out,
                'conv2_relu': self.conv2relu_out,
                'batchNorm2': self.batchNorm2_out,
                'maxpooling2': self.maxpooling2_out,
                'lstm': self.out
            }
        

# Train

In [7]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch import optim
# import time
# import copy

class ClassificationTrainer():
    def __init__(self, model, train_data, val_data, val_set, device, config):
        self.model = model
    
        self.train_data = train_data
        self.val_data = val_data
        self.val_set = val_set
        self.device = device
        self.config = config
        self.max_acc = 95.67
        self.max_val = 0
        self.train_loss_list = list()
        self.test_loss_list = list()
        self.train_acc_list = list()
        self.test_acc_list = list()
        self.val_acc_list = list()
        self.best_epoch = 0
        self.best_model = None
        self.path = self.config['result_dir'] + "buzz2_model_{}num_layers_{}filters_({},{})kernel_size_({},{})stride_({},{})maxpool_{}lr_{}epoch_{}batch_size.pt".format(
            self.config['num_layers'],
            self.config['num_filters'],
            self.config["kernel_size_1"],
            self.config["kernel_size_2"],
            self.config["stride_1"],
            self.config["stride_2"],
            self.config["maxpool_1"],
            self.config["maxpool_2"],
            self.config['lr'],
            self.config['num_epoch'],
            self.config['batch_size']
        )
    
    def train_epoch(self, criterion, opt, epoch):
        train_loss = 0.0
        train_acc = 0.0
        self.model.train()
        lenght = 0
#         print('Training epoch: ', epoch)
        for i, data in enumerate(self.train_data):
            input = data['input'].float().to(self.device)            
            target = data['labels'].float().to(self.device)
            out = self.model(input)
            lenght += len(input)
            opt.zero_grad()
            loss = criterion(out, target)
            loss.backward()
            opt.step()
            train_loss += loss.item()
            
        
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.train_data):
                input = data['input'].float().to(self.device)            
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)
            
                train_acc += torch.sum(classes == target_)
        self.train_loss = train_loss/lenght
        self.train_acc = train_acc.item()*100/lenght
        self.train_loss_list.append(self.train_loss)
        self.train_acc_list.append(self.train_acc)
        
        
#         print('\nEpoch: {}\t| Train_loss: {:.6f}\t|Train accuracy: {:.6f}'.format(epoch, self.train_loss, train_acc))
        if self.val_data is None:
            if train_loss < self.min_loss:
                self.min_loss = train_loss
                self.best_epoch = epoch
        else:
            self.validate_epoch(criterion, opt, epoch)
        
    def validate_epoch(self, criterion, opt, epoch):
        test_loss = 0.0
        test_acc = 0.
        val_acc = 0.0
        lenght = 0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_data):
                input = data['input'].float().to(self.device)
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                
                lenght += len(input)
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)
                test_acc += torch.sum(classes == target_)
                loss = criterion(out, target)
                test_loss += loss.item()
            
            for i, data in enumerate(self.val_set):
                input = data['input'].float().to(self.device)
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)
                
                val_acc += torch.sum(classes == target_)
            
        val_acc = val_acc*100/3000    
        self.val_acc_list.append(val_acc)
        test_loss = test_loss/lenght
        test_acc = test_acc.item()*100/lenght
        self.test_loss_list.append(test_loss)
        self.test_acc_list.append(test_acc)
        print('\rEpoch: {}\t| Train_loss: {:.6f}\t|Train accuracy: {:.6f}\t| Test_loss: {:.6f}\t|Test accuracy: {:.6f}\t|Val accuracy: {:.6f} '.format(epoch, self.train_loss, self.train_acc, test_loss, test_acc, val_acc), end="\r") 
        if epoch == self.config['num_epoch']:
            self.config['val_acc'] = val_acc
        wandb.log({'train_loss': self.train_loss, 'test_loss': test_loss})
        if val_acc > self.max_val: #and test_acc > self.max_acc:
            self.max_val = val_acc
            self.best_epoch = epoch
        # if test_acc > 95.67:
            self.best_model_path = self.config['result_dir'] + "buzz2_model_{}num_layers_{}filters_({},{})kernel_size_({},{})stride_({},{})maxpool_{}lr_{}epoch_{}batch_size.pt".format(
                self.config['num_layers'],
                self.config['num_filters'],
                self.config["kernel_size_1"],
                self.config["kernel_size_2"],
                self.config["stride_1"],
                self.config["stride_2"],
                self.config["maxpool_1"],
                self.config["maxpool_2"],
                self.config['lr'],
                epoch,
                self.config['batch_size']
            )
            self.best_model = copy.deepcopy(self.model)
    def train(self):
        
        self.model.to(self.device)  
        
        start = time.perf_counter()
        model_opt = optim.Adam(self.model.parameters(), self.config['lr'], weight_decay = self.config['weight_decay'])
        criterion = nn.CrossEntropyLoss()
        print("-----START TRAINING CLASSIFICATION-----")
        for epoch in range(1, self.config['num_epoch'] + 1):
            self.train_epoch(criterion, model_opt, epoch)  
        self.config['train_time'] = (time.perf_counter() - start)/60
        print("\n")
        print("----COMPLETED TRAINING CLASSIFICATION-----")
        self.config["best_epoch"] = self.best_epoch
        print('val_acc', self.config['val_acc'])
        self.config['val_acc'] = self.max_val
        wandb.config.val_acc = self.max_val
        torch.save(self.model.state_dict(), self.path)
        if self.best_model is not None:
            torch.save(self.best_model.state_dict(), self.best_model_path)
        self.config['best_model_path'] = self.best_model_path
        self.config['train_loss'] = self.train_loss_list
        self.config['train_acc'] = self.train_acc_list
        self.config['test_loss'] = self.test_loss_list
        self.config['test_accuracy'] = self.test_acc_list
        self.config['model_path'] = self.path 
#         self.config['val_acc'] = self.val_acc_list[-1]
        # self.save_loss()

    def get_updated_config(self):
        return self.config
    
    def load_model(self, path = None):
#         if path is None:
#             path = self.config['result_dir'] + "model.pt"
        self.model.load_state_dict(torch.load(path))
        self.model.eval()        
      

# Train.py

In [10]:
import torch
import sys
import time
import numpy as np
import pandas as pd

from torch.utils.data.dataloader import DataLoader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

def trainwandb():
    with wandb.init(config=config_wbd):
        config = wandb.config
        # train_set = CustomDataset(config, 'buzz1_train_data.npz')

        # data_dataloader = DataLoader(train_set, 
        #                               batch_size = 128,
        #                               shuffle = True)
        # train_dataloader = []
        # val_dataloader = []
        # for i, batch in enumerate(data_dataloader):
        #     if i < len(data_dataloader)*0.7:
        #         train_dataloader.append(batch)
        #     else:
        #         val_dataloader.append(batch)

        #buzz2
        train_set = CustomDataset(config, 'buzz2_train_data.npz')
        train_dataloader = DataLoader(train_set, 
                                    batch_size = 128,
                                    shuffle = True)
        val_set = CustomDataset(config, 'buzz2_test_training_data.npz')
        val_dataloader = DataLoader(val_set,
                                    batch_size = 256,
                                    shuffle = True)
        ###
        val_set = CustomDataset(config, 'buzz2_val_data.npz')
        val_set_ = DataLoader(val_set,
                                    batch_size = 128,
                                    shuffle = True)
        torch.manual_seed(10)
        classification_model = Conv_lstm(l_data=20000, 
                                    num_filters=config['num_filters'], 
                                    kernel_size=(config['kernel_size_1'], config['kernel_size_2']), 
                                    strides=(config['stride_1'], config['stride_2']), 
                                    maxpooling=(config['maxpool_1'], config['maxpool_2']), 
                                    d_hidden=config['d_hidden'], 
                                    num_layers= config['num_layers'],
                                    dropout=config['dropout'])
        classification_model.float()
        classification_trainer = ClassificationTrainer(model=classification_model,
                                                    train_data=train_dataloader,
                                                    val_data=val_dataloader,
                                                    val_set=val_set_,
                                                    device=device,
                                                    config=config)
        classification_trainer.train()
        config = classification_trainer.get_updated_config()
        val_acc = config['val_acc']
        wandb.log({'val_acc': val_acc})
        
        if val_acc >= 95.67:
            wandb.alert(
                title="MaiAnh found new high accuracy ID 8p4zr03y", 
                text=f"Accuracy {val_acc} "
            )
#         filename = "{}num_layers_({},{})kernel_size_{}n_filters_({},{})stride_({},{})maxpool_{}lr_{}batch_size".format(
#             config["num_layers"],
#             config["kernel_size_1"],
#             config["kernel_size_2"],
#             config["num_filters"],
#             config["stride_1"],
#             config["stride_2"],
#             config["maxpool_1"],
#             config["maxpool_2"],
#             config['lr'],
#             config['batch_size']
#         ).replace(".", "_")
#         PATH = os.path.join("configs_wandb/", "{}.yml".format(filename))
#         with open(PATH, 'w') as outfile:
#             yaml.dump(config, outfile, default_flow_style=False)
        print('ok')

In [9]:
sweep_id = wandb.sweep(sweep=config_wdb, project='quanh')

Create sweep with ID: 8p4zr03y
Sweep URL: https://wandb.ai/maiquanh/quanh/sweeps/8p4zr03y


In [ ]:
wandb.agent(sweep_id, trainwandb, count=1000)

wandb: Agent Starting Run: mtg6rr9o with config:
wandb: 	batch_size: 128
wandb: 	d_hidden: 3
wandb: 	data_dir: /home/quanhhh/Documents/model/pickle/
wandb: 	dropout: 0.1402483049603457
wandb: 	kernel_size_1: 67
wandb: 	kernel_size_2: 8
wandb: 	lr: 0.0015215192975715753
wandb: 	maxpool_1: 5
wandb: 	maxpool_2: 7
wandb: 	num_epoch: 95
wandb: 	num_filters: 25
wandb: 	num_layers: 3
wandb: 	result_dir: /home/quanhhh/Documents/model/result_wandb/
wandb: 	stride_1: 3
wandb: 	stride_2: 8
wandb: 	weight_decay: 0.00010513843648940776


-----START TRAINING CLASSIFICATION-----
Epoch: 95	| Train_loss: 0.005599	|Train accuracy: 99.485624	| Test_loss: 0.003128	|Test accuracy: 95.797599	|Val accuracy: 91.000000 

----COMPLETED TRAINING CLASSIFICATION-----
val_acc 91.0
ok


test_loss,█▇▅▄▄▃▂▄▃▃▂▃▃▂▁▁▃▂▃▃▃▃▂▃▂▁▁▂▂▃▂▁▁▁▁▁▁▁▁▁
train_loss,█▆▄▂▂▂▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁
test_loss,0.00313
train_loss,0.0056
val_acc,93.3


wandb: Agent Starting Run: e73psecd with config:
wandb: 	batch_size: 128
wandb: 	d_hidden: 3
wandb: 	data_dir: /home/quanhhh/Documents/model/pickle/
wandb: 	dropout: 0.1833942380174814
wandb: 	kernel_size_1: 68
wandb: 	kernel_size_2: 6
wandb: 	lr: 0.0018991473049144764
wandb: 	maxpool_1: 6
wandb: 	maxpool_2: 4
wandb: 	num_epoch: 97
wandb: 	num_filters: 31
wandb: 	num_layers: 2
wandb: 	result_dir: /home/quanhhh/Documents/model/result_wandb/
wandb: 	stride_1: 5
wandb: 	stride_2: 6
wandb: 	weight_decay: 0.00013201483033910028


-----START TRAINING CLASSIFICATION-----
Epoch: 97	| Train_loss: 0.005661	|Train accuracy: 97.942495	| Test_loss: 0.003477	|Test accuracy: 82.246998	|Val accuracy: 73.233330 

----COMPLETED TRAINING CLASSIFICATION-----
val_acc 73.23332977294922
ok


...

wandb: Agent Starting Run: 191noc85 with config:
wandb: 	batch_size: 128
wandb: 	d_hidden: 3
wandb: 	data_dir: /home/quanhhh/Documents/model/pickle/
wandb: 	dropout: 0.1504667118819048
wandb: 	kernel_size_1: 86
wandb: 	kernel_size_2: 10
wandb: 	lr: 0.0003650327075865867
wandb: 	maxpool_1: 8
wandb: 	maxpool_2: 4
wandb: 	num_epoch: 99
wandb: 	num_filters: 49
wandb: 	num_layers: 3
wandb: 	result_dir: /home/quanhhh/Documents/model/result_wandb/
wandb: 	stride_1: 3
wandb: 	stride_2: 3
wandb: 	weight_decay: 0.0002276592128066813


-----START TRAINING CLASSIFICATION-----
Epoch: 99	| Train_loss: 0.005872	|Train accuracy: 88.301240	| Test_loss: 0.003880	|Test accuracy: 74.228130	|Val accuracy: 85.566666 

----COMPLETED TRAINING CLASSIFICATION-----
val_acc 85.56666564941406
ok


test_loss,██▇▆▅▅▅▅▅▄▅▄▄▃▃▂▂▂▂▂▁▂▂▁▃▁▁▁▁▁▁▁▁▁▁▁▁▂▁▄
train_loss,██▆▆▅▄▄▄▄▄▃▃▃▃▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁
test_loss,0.00388
train_loss,0.00587
val_acc,96.26666


wandb: Agent Starting Run: hdff717y with config:
wandb: 	batch_size: 128
wandb: 	d_hidden: 3
wandb: 	data_dir: /home/quanhhh/Documents/model/pickle/
wandb: 	dropout: 0.1647682081841993
wandb: 	kernel_size_1: 81
wandb: 	kernel_size_2: 7
wandb: 	lr: 0.0003541196588349586
wandb: 	maxpool_1: 5
wandb: 	maxpool_2: 5
wandb: 	num_epoch: 90
wandb: 	num_filters: 57
wandb: 	num_layers: 3
wandb: 	result_dir: /home/quanhhh/Documents/model/result_wandb/
wandb: 	stride_1: 6
wandb: 	stride_2: 7
wandb: 	weight_decay: 0.00016109075997227133


-----START TRAINING CLASSIFICATION-----
